In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from pykalman import KalmanFilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from statsmodels.tsa.stattools import adfuller

# Load dataset
data = pd.read_csv('../datasets/CropSDEData/METEO_DEKADS_NUTS2_NL.csv')

# Feature Selection
features = ['TAVG', 'VPRES', 'WSPD', 'RELH']
target = 'PREC'

# Drop rows with missing values
data = data.dropna(subset=features + [target])

# Prepare data
X = data[features]
y = data[target]

# Ensure stationarity of the target variable
if adfuller(y)[1] > 0.05:
    print("Target variable is non-stationary. Applying differencing...")
    y = y.diff().dropna()
    X = X.iloc[1:]  # Align X with y after differencing

# Apply the same transformation to features
X = X.diff().dropna()

# Align X and y to ensure consistent lengths
if len(X) > len(y):
    X = X.iloc[:len(y)]
elif len(y) > len(X):
    y = y.iloc[:len(X)]

# Scale the Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Kalman Filter for Volatility Estimation
def estimate_volatility_kalman(y):
    kf = KalmanFilter(
        transition_matrices=[1],
        observation_matrices=[1],
        initial_state_mean=np.var(y),
        initial_state_covariance=1e-4,
        transition_covariance=1e-5,
        observation_covariance=1e-2
    )
    state_means, _ = kf.filter(y**2)
    return np.sqrt(np.maximum(state_means.flatten(), 1e-5))  # Ensure non-negativity

# Estimate volatility from the observed data
volatility_estimates = estimate_volatility_kalman(y.values)

# Maximum Likelihood Estimation (MLE) for Heston Model
def heston_log_likelihood(params, data, vol_estimates):
    alpha, beta, kappa, theta, xi = params
    dt = 1  # Assuming daily intervals

    log_likelihood = 0
    for t in range(1, len(data)):
        V_t = vol_estimates[t]  # Kalman-filtered volatility
        residual = data[t] - (data[t-1] + alpha * (beta - data[t-1]) * dt)
        log_likelihood += -0.5 * np.log(2 * np.pi * V_t * dt) - (residual**2) / (2 * V_t * dt)

    return -log_likelihood  # Negative for minimization

# Initial guesses and bounds for MLE
initial_guess = [0.1, np.mean(y), 0.5, np.var(y), 0.5]
bounds = [(1e-5, None), (None, None), (1e-5, None), (1e-5, None), (1e-5, None)]

# Perform MLE for Heston Model
res_mle = minimize(heston_log_likelihood, initial_guess, args=(y.values, volatility_estimates), method='L-BFGS-B', bounds=bounds)

if res_mle.success:
    alpha_mle, beta_mle, kappa_mle, theta_mle, xi_mle = res_mle.x
    print("\nEstimated Heston Parameters using Maximum Likelihood Estimation (MLE):")
    print(f"Alpha: {alpha_mle}, Beta: {beta_mle}, Kappa: {kappa_mle}, Theta: {theta_mle}, Xi: {xi_mle}")
else:
    print("\nMLE failed to converge.")
    print(f"Message: {res_mle.message}")
    exit()

# Scale Heston Parameters for Stability
alpha_mle /= 10
beta_mle /= 10
theta_mle /= 10

# Define Neural Network model with Heston-Informed Weights
class HestonNN(nn.Module):
    def __init__(self, input_size, alpha, beta, theta):
        super(HestonNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.batchnorm1 = nn.BatchNorm1d(32)

        # Initialize weights using Heston parameters
        nn.init.normal_(self.fc1.weight, mean=beta, std=np.sqrt(theta))
        nn.init.constant_(self.fc1.bias, alpha)
        nn.init.normal_(self.fc2.weight, mean=beta, std=np.sqrt(theta))
        nn.init.constant_(self.fc2.bias, alpha)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize Neural Network
input_size = X_scaled.shape[1]
model = HestonNN(input_size, alpha_mle, beta_mle, theta_mle)

# Define loss, optimizer, and learning rate scheduler
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y.values, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Training loop
epochs = 500
early_stopping_patience = 50
best_loss = np.inf
patience_counter = 0

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    scheduler.step()

    if loss.item() < best_loss:
        best_loss = loss.item()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter > early_stopping_patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor).numpy()

test_mse = mean_squared_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)

print(f"\nNeural Network Test MSE: {test_mse}")
print(f"Neural Network Test R^2 Score: {test_r2}")

# Compare with Linear Regression
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
lr_mse = mean_squared_error(y_test, y_pred_lr)
lr_r2 = r2_score(y_test, y_pred_lr)

print(f"\nLinear Regression Test MSE: {lr_mse}")
print(f"Linear Regression Test R^2 Score: {lr_r2}")

if test_mse < lr_mse:
    print("\nNeural Network outperforms Linear Regression.")
else:
    print("\nLinear Regression outperforms Neural Network.")


Estimated Heston Parameters using Maximum Likelihood Estimation (MLE):
Alpha: 0.9670405461079659, Beta: 1.8822030558211882, Kappa: 0.5, Theta: 2.5673585008286097, Xi: 0.5
Epoch [50/500], Loss: 16.5954
Epoch [100/500], Loss: 11.1984
Epoch [150/500], Loss: 9.6076
Epoch [200/500], Loss: 8.8410
Epoch [250/500], Loss: 8.6208
Epoch [300/500], Loss: 8.3158
Epoch [350/500], Loss: 8.3665
Early stopping at epoch 369

Neural Network Test MSE: 5.34693564142795
Neural Network Test R^2 Score: -1.1176418339305583

Linear Regression Test MSE: 2.107747312976005
Linear Regression Test R^2 Score: 0.16523141764977267

Linear Regression outperforms Neural Network.
